In [26]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [27]:
INITIAL_REASONING_PROMPT_TEMPLATE = """
Generate an initial reasoning step using placeholders for the specific values and function calls needed. 
The goal is to determine the first function to call based on the question provided and the available functions.

Use the placeholders labeled y1, y2, etc., to represent outputs if needed. 
The reasoning should lead to only one function call, represented as an inline string like [FUNC {{function_name}}({{input1}}, {{input2}}, ...) = {{output_placeholder}}].

You are not required to use all functions, but you must use at least one function that best matches the question's intent. 
If the question can be answered without any function, you can conclude so.

Assume someone will read the plan after this function has been executed to continue further steps.

Example:
-----------
Available functions:
```python
def add(a: int, b: int) -> int:
    \"\"\"Add two numbers together.\"\"\"
    ...

def multiply(a: int, b: int) -> int:
    \"\"\"Multiply two numbers together.\"\"\"
    ...
```   
    
Question:
Sally has 3 apples and buys 2 more. Then magically, a wizard casts a spell that multiplies the number of apples by 3. How many apples does Sally have now?

Abstract plan of reasoning:
After buying the apples, Sally has [FUNC add(3, 2) = y1] apples.

Your Turn:
-----------
Available functions:
```python
{functions}
```

Question:
{question}

Abstract plan of reasoning:
"""

NEXT_STEP_REASONING_PROMPT_TEMPLATE = """
Based on the outputs generated from the previous function calls, decide whether additional steps are necessary:

If no further steps are required, return "NO".
If further steps are needed, provide a clear reasoning step using placeholders (y1, y2, etc.) for specific values and function calls. The objective is to determine the next function to call, using available outputs and aligning with the intent of the question.
Use placeholders (y1, y2, etc.) to represent the outputs of previous function calls if needed.
The reasoning should lead to a single, well-justified function call, represented as: [FUNC {function_name}({input1}, {input2}, ...) = {output_placeholder}].
You are not required to use all functions, but you must select at least one function that best matches the question's intent or utilizes keywords from the previous output to refine the results.
If the question can be fully answered without invoking additional functions, conclude accordingly.
Assumptions:
Assume that someone will read this plan after executing the current function to determine the next steps.

Example:
-----------
Available functions:
```python
def add(a: int, b: int) -> int:
    \"\"\"Add two numbers together.\"\"\"
    ...

def multiply(a: int, b: int) -> int:
    \"\"\"Multiply two numbers together.\"\"\"
    ...
```   
Question:
Sally has 3 apples and buys 2 more. Then magically, a wizard casts a spell that multiplies the number of apples by 3. How many apples does Sally have now?

Previous Function Call Output:
After buying the apples, Sally has [FUNC add(3, 2) = y1] apples. After executing the first function, we have y1 = 5.

Abstract Plan of Reasoning for Next Step:
Since a wizard casts a spell that multiplies the number of apples by 3, use the output from the previous function call to determine the total apples. Thus, the next function call is [FUNC multiply(y1, 3)].



Your Turn:
-----------
Available functions:
```python
{functions}
```

Question:
{question}

Previous Function Call Output:
{function_call_output}

Abstract plan of reasoning for next step:
"""

In [28]:
from llama_index.core.tools.types import AsyncBaseTool
from typing import List, Optional
from llama_index.core.tools import (
    BaseTool,
    FunctionTool,
    ToolOutput,
    adapt_to_async_tool,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core.base.llms.types import ChatMessage
from IPython.display import display, Markdown
from agents.coa.tools_handler.tool_retriver import ToolRetriever

In [29]:
# main.py
import asyncio
from typing import Dict
from pydantic import BaseModel, Field
from agents.pdf_reader_agent.workflow import ConciergeWorkflow as DietConsultantAgent, ConciergeWorkflow as NutritionConsultantAgent


# Define a Pydantic model for input validation
class DietQuery(BaseModel):
    query: str = Field(description="A question or query related to diet.")

# Asynchronous function with Pydantic validation
async def consult_diet_async(query: str) -> Dict[str, str]:
    concierge = DietConsultantAgent(timeout=180, verbose=True)
    result = await concierge.run(query=query, collection_name="pdf-diet-docs")
    return result

# Synchronous wrapper for the asynchronous function with Pydantic validation
def consult_diet(query: str) -> Dict[str, object]:
    # Validate the input using the Pydantic model
    input_data = DietQuery(query=query)
    # Run the asynchronous function
    return asyncio.run(consult_diet_async(input_data.query))


# Define a Pydantic model for input validation with the updated class name
class NutritionQuery(BaseModel):
    query: str = Field(description="A question or query related to nutrition.")

# Asynchronous function with Pydantic validation and updated collection name
async def consult_nutrition_async(query: str) -> Dict[str, str]:
    concierge = NutritionConsultantAgent(timeout=180, verbose=True)
    result = await concierge.run(query=query, collection_name="pdf-nutrition-docs")
    return result

# Synchronous wrapper for the asynchronous function with Pydantic validation
def consult_nutrition(query: str) -> Dict[str, object]:
    # Validate the input using the updated Pydantic model
    input_data = NutritionQuery(query=query)
    # Run the asynchronous function
    return asyncio.run(consult_nutrition_async(input_data.query))


diet_tool = FunctionTool.from_defaults(
        fn=consult_diet,
        name="consult_diet",
        description="Consults on diet based on a query.",
    )

# Create the FunctionTool for nutrition consultation with the synchronous wrapper
nutrition_tool = FunctionTool.from_defaults(
    fn=consult_nutrition,
    name="consult_nutrition",
    description="Provides nutritional information based on a query.",
)

tools = [diet_tool, nutrition_tool]

In [30]:
tool_retriever = ToolRetriever(tools)
query = (
        "How do omega-3 fatty acids found in fish impact brain health, and what are the recommended dietary guidelines "
        "for incorporating fish into a Mediterranean diet to maximize cognitive benefits? "
        "Please include both the nutritional science behind omega-3s and practical meal planning tips."
        "What is the Indo-Mediterranean diet"
    )

# query = ("What is the Indo-Mediterranean diet.")

# query = ("What is the Indo-Mediterranean diet?")
retrieved_tools = tool_retriever.prepare_tools("Mediterranean Diet")
print(f"==== Available Parsed Functions ====")
for tool_str in retrieved_tools["tools_strs"]:
    print(tool_str)
    
tools_strs = retrieved_tools["tools_strs"]


==== Available Parsed Functions ====
def consult_diet(query: string):
   """Consults on diet based on a query."""
    ...
def consult_nutrition(query: string):
   """Provides nutritional information based on a query."""
    ...


In [31]:
reasoning_prompt = INITIAL_REASONING_PROMPT_TEMPLATE.format(
    functions="\n".join(tools_strs), question=query
)
# display(Markdown(reasoning_prompt))
# messages = []
gpt4o = OpenAI(temperature=0, model="gpt-4o")
Settings.llm = gpt4o

In [32]:
reasoning_message = ChatMessage(role="user", content=reasoning_prompt)
# messages.append(reasoning_message)

# run the reasoning prompt
response = await gpt4o.achat([reasoning_message])

In [33]:
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Abstract plan of reasoning:\nTo address the question about the impact of omega-3 fatty acids on brain health and the recommended dietary guidelines for incorporating fish into a Mediterranean diet, we need to first gather nutritional information about omega-3 fatty acids. Therefore, we should start by consulting the nutritional science behind omega-3s. This can be achieved by calling the function that provides nutritional information based on a query.\n\nInitial reasoning step:\nTo understand the nutritional science behind omega-3 fatty acids, we will use [FUNC consult_nutrition("omega-3 fatty acids and brain health") = y1].', additional_kwargs={}), raw=ChatCompletion(id='chatcmpl-A23YgJynf05MbAAkkq3FF3D0GAJGI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Abstract plan of reasoning:\nTo address the question about the impact of omega-3 fatty acids

In [34]:
print(str(response.message.content))
solution = response.message.content

Abstract plan of reasoning:
To address the question about the impact of omega-3 fatty acids on brain health and the recommended dietary guidelines for incorporating fish into a Mediterranean diet, we need to first gather nutritional information about omega-3 fatty acids. Therefore, we should start by consulting the nutritional science behind omega-3s. This can be achieved by calling the function that provides nutritional information based on a query.

Initial reasoning step:
To understand the nutritional science behind omega-3 fatty acids, we will use [FUNC consult_nutrition("omega-3 fatty acids and brain health") = y1].


In [35]:
import re
func_calls = re.findall(r"\[FUNC (\w+)\((.*?)\) = (\w+)\]", solution)
func_call = func_calls[0]

In [36]:
func_name, raw_inputs, output_placeholder = func_call
raw_inputs

'"omega-3 fatty acids and brain health"'

In [37]:
input_data = []
results = {}
for raw_input in raw_inputs.split(","):
    raw_input = raw_input.strip()
    try:
        input_data.append(int(results.get(raw_input, raw_input)))
    except ValueError:
        input_data.append(raw_input)
        
input_data

['"omega-3 fatty acids and brain health"']

In [38]:
tools_by_name = retrieved_tools["tools_by_name"]
tools_by_name

{'consult_diet': <llama_index.core.tools.function_tool.FunctionTool at 0x28e7502d0>,
 'consult_nutrition': <llama_index.core.tools.function_tool.FunctionTool at 0x28e778cd0>}

In [39]:
tool_output = await tools_by_name[func_name].acall(*input_data)

In [40]:
tool_output.raw_output

{'query_result': 'Omega-3 fatty acids have been shown to play a significant role in brain health. Dietary supplementation with omega-3 fatty acids, particularly docosahexaenoic acid (DHA), has been found to elevate levels of hippocampal brain-derived neurotrophic factor (BDNF) and enhance cognitive function. These fatty acids may improve cognitive abilities by facilitating synaptic plasticity, enhancing synaptic membrane fluidity, stimulating glucose utilization, and improving mitochondrial function, thereby reducing oxidative stress. Studies have also indicated that omega-3 fatty acid supplementation can reduce cognitive deficits associated with psychiatric disorders and improve school performance in children.',
 'source_node': ['Incorporating this knowledge into the design of novel treatments could be vital to combating mental diseases and neurological weaknesses. Acknowledgments In the elaboration of this article, I greatly benefited from inspirational discussions with G. Cole, I. C

In [41]:
from agents.coa_agent.prompts.evaluator import prometheus_relevancy_eval_prompt_template, prometheus_relevancy_refine_prompt_template
from agents.coa_agent.validator.relevancy_eval import GPT4RelevancyEvaluator
import nest_asyncio
nest_asyncio.apply()



In [42]:
relevancy_eval_prompt_template = """###Task Description: An instruction (might include an Input inside it), a query, context, and a score rubric representing evaluation criteria are given. 
       1. You are provided with evaluation task with the help of a query and context output by function tool.
       2. Write a detailed feedback based on evaluation task and the given score rubric, not evaluating in general. 
       3. After writing a feedback, write a score that is YES or NO. You should refer to the score rubric. 
       4. The output format should look as follows: "Feedback: (write a feedback for criteria) [RESULT] (YES or NO)” 
       5. Please do not generate any other opening, closing, and explanations. 

        ###The instruction to evaluate: Your task is to evaluate if the source nodes for the query whether they are in line with the context information provided.

        ###Query: {query_str} 

        ###Context: {context_str}
            
        ###Score Rubrics: 
        Score YES: If the context information provided is sufficient and in line to answer the query.
        Score NO: If the context information provided is sufficient and in line to answer the query.
        
        ###Feedback: """

In [43]:
evaluator = GPT4RelevancyEvaluator(relevancy_eval_prompt_template, prometheus_relevancy_refine_prompt_template)

result_source_node = evaluator.evaluate_sources(query, tool_output.raw_output)
    



The context provided in the query is not sufficient to answer the question. The query asks for information on the impact of omega-3 fatty acids on brain health and practical meal planning tips for incorporating fish into a Mediterranean diet. However, the context does not provide any relevant information on these topics. It only mentions the Indo-Mediterranean diet and acknowledges the contributions of various individuals and organizations. This does not align with the query's focus on omega-3 fatty acids, brain health, and dietary guidelines. Therefore, the context information provided is not sufficient and in line to answer the query. 

The score rubric requires the context information to be sufficient and in line to answer the query. In this case, the context information is not sufficient and does not align with the query's focus. Hence, the score is NO. 

[RESULT] NO


The context provided is comprehensive and directly addresses the query. It delves into the nutritional science b

In [44]:
print(f"==== Available Parsed Functions ====")
for tool_str in retrieved_tools["tools_strs"]:
    print(tool_str)
    
parsed_functions = "==== Available Parsed Functions ==== \n"
for tool_str in retrieved_tools["tools_strs"]:
    parsed_functions += tool_str


==== Available Parsed Functions ====
def consult_diet(query: string):
   """Consults on diet based on a query."""
    ...
def consult_nutrition(query: string):
   """Provides nutritional information based on a query."""
    ...


In [45]:
accumulated_source_nodes = []
accumulated_source_nodes.append(result_source_node)
results[output_placeholder] = result_source_node
results

{'y1': (['Gómez-Pinilla Page 7 which DHA affects brain plasticity and cognition are starting to be elucidated. For example, DHA dietary supplementation has been found to elevate levels of hippocampal BDNF and enhance cognitive function in rodent models of brain trauma81. DHA might enhance cognitive abilities by facilitating synaptic plasticity and/or enhancing synaptic membrane fluidity; it might also act through its effects on metabolism, as DHA stimulates glucose utilization82 and mitochondrial function83, reducing oxidative stress (OS)81. Most of the studies in humans have been directed at evaluating the effects of omega-3 fatty acids on reducing the cognitive deficit that is associated with psychiatric disorders. Several other, widely publicized, attempts to determine the effects of omega-3 fatty acid supplementation on the performance of school children have been carried out. A randomized double-blind controlled trial in which half of the children received omega-3 fatty acids and 

In [48]:
accumulated_solution = []
for placeholder, value in results.items():
    solution = solution.replace("Abstract plan of reasoning", "").replace(f"{placeholder}", '"' + str(value) + '"')
    print("\n")
    
accumulated_solution.append(solution)


In [49]:
solution

':\nTo address the question about the impact of omega-3 fatty acids on brain health and the recommended dietary guidelines for incorporating fish into a Mediterranean diet, we need to first gather nutritional information about omega-3 fatty acids. Therefore, we should start by consulting the nutritional science behind omega-3s. This can be achieved by calling the function that provides nutritional information based on a query.\n\nInitial reasoning step:\nTo understand the nutritional science behind omega-3 fatty acids, we will use [FUNC consult_nutrition("omega-3 fatty acids and brain health") = "([\'Gómez-Pinilla Page 7 which DHA affects brain plasticity and cognition are starting to be elucidated. For example, DHA dietary supplementation has been found to elevate levels of hippocampal BDNF and enhance cognitive function in rodent models of brain trauma81. DHA might enhance cognitive abilities by facilitating synaptic plasticity and/or enhancing synaptic membrane fluidity; it might al

In [50]:
display(Markdown(solution))

:
To address the question about the impact of omega-3 fatty acids on brain health and the recommended dietary guidelines for incorporating fish into a Mediterranean diet, we need to first gather nutritional information about omega-3 fatty acids. Therefore, we should start by consulting the nutritional science behind omega-3s. This can be achieved by calling the function that provides nutritional information based on a query.

Initial reasoning step:
To understand the nutritional science behind omega-3 fatty acids, we will use [FUNC consult_nutrition("omega-3 fatty acids and brain health") = "(['Gómez-Pinilla Page 7 which DHA affects brain plasticity and cognition are starting to be elucidated. For example, DHA dietary supplementation has been found to elevate levels of hippocampal BDNF and enhance cognitive function in rodent models of brain trauma81. DHA might enhance cognitive abilities by facilitating synaptic plasticity and/or enhancing synaptic membrane fluidity; it might also act through its effects on metabolism, as DHA stimulates glucose utilization82 and mitochondrial function83, reducing oxidative stress (OS)81. Most of the studies in humans have been directed at evaluating the effects of omega-3 fatty acids on reducing the cognitive deficit that is associated with psychiatric disorders. Several other, widely publicized, attempts to determine the effects of omega-3 fatty acid supplementation on the performance of school children have been carried out. A randomized double-blind controlled trial in which half of the children received omega-3 fatty acids and the other half received placebos is being conducted across several schools in Durham, UK84. Previous studies from the same investigators showed that omega-3 fatty acid supplementation was associated with reduced cognitive deficits (in reading and spelling, and teaching-rated behaviour) in children affected with developmental coordination disorder — that is, in children with specific impairments of motor function that are independent of their motor ability 85. In the new studies, children were selected on the basis that “they were not fulfilling their potential at school” but their general ability was deemed “normal”, and they were subjected to regular tests to measure their coordination, concentration and academic ability. According to preliminary results84, some level of improvement in school performance was observed in the group receiving omega-3 fatty acids, unleashing a flurry of speculations from the media. Although the results of the Durham study require scientific scrutiny for validation, they seem to agree with those of another study86 in which omega-3 fatty acids (DHA 88 mg per day and eicosapentaenoic acid (EPA) 22 mg per day) and micro-nutrients (iron, zinc, folate and vitamins A, B6, B12 and C) were provided in a drink mix to 396 children (6–12 years of age) in Australia and 394 children in Indonesia. The results showed higher scores on tests that measured verbal intelligence and learning and memory after 6 and 12 months in both boys and girls in Australia, but in only girls in Indonesia.'], 1)"].

In [51]:
reasoning_prompt = NEXT_STEP_REASONING_PROMPT_TEMPLATE.format(
    functions="\n".join(tools_strs), question=query, function_call_output=solution
)

reasoning_message = ChatMessage(role="user", content=reasoning_prompt)
# messages.append(reasoning_message)

# run the reasoning prompt
response = await gpt4o.achat([reasoning_message])

KeyError: 'function_name'

In [52]:
display(Markdown(reasoning_prompt))


Generate an initial reasoning step using placeholders for the specific values and function calls needed. 
The goal is to determine the first function to call based on the question provided and the available functions.

Use the placeholders labeled y1, y2, etc., to represent outputs if needed. 
The reasoning should lead to only one function call, represented as an inline string like [FUNC {function_name}({input1}, {input2}, ...) = {output_placeholder}].

You are not required to use all functions, but you must use at least one function that best matches the question's intent. 
If the question can be answered without any function, you can conclude so.

Assume someone will read the plan after this function has been executed to continue further steps.

Example:
-----------
Available functions:
```python
def add(a: int, b: int) -> int:
    """Add two numbers together."""
    ...

def multiply(a: int, b: int) -> int:
    """Multiply two numbers together."""
    ...
```   
    
Question:
Sally has 3 apples and buys 2 more. Then magically, a wizard casts a spell that multiplies the number of apples by 3. How many apples does Sally have now?

Abstract plan of reasoning:
After buying the apples, Sally has [FUNC add(3, 2) = y1] apples.

Your Turn:
-----------
Available functions:
```python
def consult_diet(query: string):
   """Consults on diet based on a query."""
    ...
def consult_nutrition(query: string):
   """Provides nutritional information based on a query."""
    ...
```

Question:
How do omega-3 fatty acids found in fish impact brain health, and what are the recommended dietary guidelines for incorporating fish into a Mediterranean diet to maximize cognitive benefits? Please include both the nutritional science behind omega-3s and practical meal planning tips.What is the Indo-Mediterranean diet

Abstract plan of reasoning:


In [53]:
solution = response.message.content
solution

'Abstract plan of reasoning:\nTo address the question about the impact of omega-3 fatty acids on brain health and the recommended dietary guidelines for incorporating fish into a Mediterranean diet, we need to first gather nutritional information about omega-3 fatty acids. Therefore, we should start by consulting the nutritional science behind omega-3s. This can be achieved by calling the function that provides nutritional information based on a query.\n\nInitial reasoning step:\nTo understand the nutritional science behind omega-3 fatty acids, we will use [FUNC consult_nutrition("omega-3 fatty acids and brain health") = y1].'

In [54]:
import re
func_calls = re.findall(r"\[FUNC (\w+)\((.*?)\) = (\w+)\]", solution)
func_call = func_calls

In [55]:
func_call

[('consult_nutrition', '"omega-3 fatty acids and brain health"', 'y1')]

In [56]:
results

{'y1': (['Gómez-Pinilla Page 7 which DHA affects brain plasticity and cognition are starting to be elucidated. For example, DHA dietary supplementation has been found to elevate levels of hippocampal BDNF and enhance cognitive function in rodent models of brain trauma81. DHA might enhance cognitive abilities by facilitating synaptic plasticity and/or enhancing synaptic membrane fluidity; it might also act through its effects on metabolism, as DHA stimulates glucose utilization82 and mitochondrial function83, reducing oxidative stress (OS)81. Most of the studies in humans have been directed at evaluating the effects of omega-3 fatty acids on reducing the cognitive deficit that is associated with psychiatric disorders. Several other, widely publicized, attempts to determine the effects of omega-3 fatty acid supplementation on the performance of school children have been carried out. A randomized double-blind controlled trial in which half of the children received omega-3 fatty acids and 

In [57]:
func_name, raw_inputs, output_placeholder = func_call[-1]
raw_inputs, func_name, output_placeholder

('"omega-3 fatty acids and brain health"', 'consult_nutrition', 'y1')

In [58]:
results.get(raw_input, raw_input)

'"omega-3 fatty acids and brain health"'

In [59]:
input_data = []
for raw_input in raw_inputs.split(","):
    raw_input = raw_input.strip()
    try:
        input_data.append(int(results.get(raw_input, raw_input)[0][0]))
    except ValueError:
        input_data.append(results.get(raw_input, raw_input))
        
input_data

['"omega-3 fatty acids and brain health"']

In [60]:
tool_output = await tools_by_name[func_name].acall(*input_data)

In [40]:
evaluator = GPT4RelevancyEvaluator(relevancy_eval_prompt_template, prometheus_relevancy_refine_prompt_template)

result_source_node = evaluator.evaluate_sources(query, tool_output.raw_output)
results[output_placeholder] = result_source_node
results

The context provided does not contain any information about the Indo-Mediterranean diet, which is the subject of the query. The context discusses the effects of certain cytokines, the benefits of fish in a diet, and the criteria for diagnosing metabolic syndrome. However, it does not provide any information about what the Indo-Mediterranean diet is, what it includes, or any other relevant details. Therefore, it is not in line with the query. [RESULT] NO
The context provided does not contain any information about the Indo-Mediterranean diet. It mainly discusses the benefits of fish consumption and its impact on cardiovascular diseases. Therefore, it does not provide any source nodes that can be used to answer the query about the Indo-Mediterranean diet. [RESULT] NO
Number of nodes before validated  2
Number of nodes after validated  0


{'y1': (['No output is generated. Please modify the input for the current function call'],
  1)}

In [41]:
solution

'Based on the provided output, the next steps should be planned to further elaborate on the Indo-Mediterranean diet. The initial function call has provided a detailed description of the diet and its benefits. However, if the goal is to obtain more specific nutritional information or further consult on the diet, we can proceed with another function call.\n\nGiven the available functions, the next logical step would be to use the `consult_nutrition` function to get more detailed nutritional information about the Indo-Mediterranean diet.\n\nAbstract plan of reasoning for next step:\nTo obtain detailed nutritional information about the Indo-Mediterranean diet, we should use the function that provides nutritional information based on a query. The query will be "Indo-Mediterranean diet".\n\n[FUNC consult_nutrition("Indo-Mediterranean diet") = y1]'

In [42]:
next_solution = solution
for placeholder, value in results.items():
    print(value)
    next_solution = next_solution.replace(f"{placeholder}", '"' + str(value) + '"').replace("Abstract plan of reasoning for next step", "")
    print("\n")

display(Markdown(next_solution))

(['No output is generated. Please modify the input for the current function call'], 1)




Based on the provided output, the next steps should be planned to further elaborate on the Indo-Mediterranean diet. The initial function call has provided a detailed description of the diet and its benefits. However, if the goal is to obtain more specific nutritional information or further consult on the diet, we can proceed with another function call.

Given the available functions, the next logical step would be to use the `consult_nutrition` function to get more detailed nutritional information about the Indo-Mediterranean diet.

:
To obtain detailed nutritional information about the Indo-Mediterranean diet, we should use the function that provides nutritional information based on a query. The query will be "Indo-Mediterranean diet".

[FUNC consult_nutrition("Indo-Mediterranean diet") = "(['No output is generated. Please modify the input for the current function call'], 1)"]

In [27]:
reasoning_prompt = NEXT_STEP_REASONING_PROMPT_TEMPLATE.format(
    functions="\n".join(tools_strs), question=query, function_call_output=next_solution
)

reasoning_message = ChatMessage(role="user", content=reasoning_prompt)
# messages.append(reasoning_message)

# run the reasoning prompt
response = await gpt4o.achat([reasoning_message])

accumulated_solution.append(solution)

NameError: name 'next_solution' is not defined

In [28]:
display(Markdown(response.message.content))

Based on the provided output, the next steps should be planned to further elaborate on the Indo-Mediterranean diet. The initial function call has provided a detailed description of the diet and its benefits. However, if the goal is to obtain more specific nutritional information or further consult on the diet, we can proceed with another function call.

Given the available functions, the next logical step would be to use the `consult_nutrition` function to get more detailed nutritional information about the Indo-Mediterranean diet.

Abstract plan of reasoning for next step:
To obtain detailed nutritional information about the Indo-Mediterranean diet, we should use the function that provides nutritional information based on a query. The query will be "Indo-Mediterranean diet".

[FUNC consult_nutrition("Indo-Mediterranean diet") = y1]